In [ ]:
import logging
logger = logging.getLogger("spacy")
logger.setLevel(logging.ERROR)

import dask.bag as db
import json
import pandas as pd

docs = db.read_text('/home/felix/code/clairefiltz/litreview/raw_data/data_literature_review.json').map(json.loads)
# start prototyping with a subset of the data so it's easyer to handel:
# This procedure was recommended in the ArXiv dataset itself

get_latest_version = lambda x: x['versions'][-1]['created']


# get only necessary fields of the metadata file
trim = lambda x: {'id': x['id'],
                  'authors': x['authors'],
                  'title': x['title'],
                  'doi': x['doi'],
                  'category':x['categories'].split(' '),
                  'abstract':x['abstract'],}
# filter for papers published on or after 2019-01-01
columns = ['id','category','abstract']
docs_df = (docs.filter(lambda x: int(get_latest_version(x).split(' ')[3]) > 2018)
           .map(trim).
           compute())

# convert to pandas
docs_df = pd.DataFrame(docs_df)
#save trimmed dataset for later use so we can skip the dataset trimming later:
docs_df.to_csv("trimmed_arxiv_docs.csv", index=False)
docs_df.head(10)

In [10]:
docs_df.doi.isnull().sum()/len(docs_df)

0.6536969508883544

In [12]:
docs_df.category.nunique()

TypeError: unhashable type: 'list'

In [ ]:
- check for numbers of categories, category combinations
- check for missing values in: title, abstract
    - tokenize
    - bag-of-words, tdifid?, word2vec